In [1]:
from numpy import *
import numpy as np
import math
import cmath
from numpy.linalg import qr
from math import fabs

In [2]:
def LU(a,b):
    u=a
    l=np.zeros([len(b),len(b)])
    for k in range(1,len(b)):
        for i in range(k-1,len(b)):
            for j in range(i,len(b)):
                l[j,i]=u[j,i]/u[i,i]
        for i in range(k,len(b)):
            for j in range(k-1,len(b)):
                u[i,j]=u[i,j]-l[i,k-1]*u[k-1,j]
    result=(u,l)
    return result

In [3]:
def prog(a,b,c,d):
    n=len(d)
    x=np.zeros(n)
    p=np.zeros(n)
    q=np.zeros(n)
    i=0
    p[0]=-c[0]/b[0]
    q[0]=d[0]/b[0]
    for i in range(1,n-1):
        p[i]=-c[i]/(a[i]*p[i-1]+b[i])
    print('p= ',p)
    i=0
    for i in range(1,n):
        q[i]=(d[i]-a[i]*q[i-1])/(a[i]*p[i-1]+b[i])
    print('q= ',q)
    i=3
    x[n-1]=q[n-1]
    while i>=0:
        x[i]=p[i]*x[i+1]+q[i]
        i=i-1
    return x

In [4]:
def norma_matrix(a,count):
    norm=0.
    for i in range(0,count-1):
        sum_a=0
        for j in range(0,count-1):
            sum_a+=math.fabs(a[i,j])
        if sum_a>norm:
            norm=sum_a
    return norm

def norma_vector(b):
    normv=b[0]
    for i in range(1,len(b)-1):
        if math.fabs(b[i])>normv:
            normv=math.fabs(b[i])
    return normv

def method_yakobi(a,b,count):
    alfa=np.zeros((count,count))
    beta=np.zeros(count)
    for i in range(0,count):
        beta[i]=b[i]/a[i,i]
        for j in range(0,count):
            if i==j:
                alfa[i,j]=0
            else:
                alfa[i,j]=-a[i,j]/a[i,i]
    res=(alfa,beta)
    return res

def matr_vect(a,b):
    res=np.zeros(len(b))
    for i in range(0,len(b)):
        for j in range(0,len(b)):
            res[i]+=a[i,j]*b[j]
    return res

def vect_p_vect(b,c):
    res=np.zeros(len(b))
    for i in range(0,len(b)):
        res[i]=b[i]+c[i]
    return res

def vect_m_vect(b,c):
    res=np.zeros(len(b))
    for i in range(0,len(b)):
        res[i]=b[i]-c[i]
    return res

def sum_razl(a,count):
    b=np.zeros((count,count))
    c=np.zeros((count,count))
    for i in range(0,count):
        for j in range(0,count):
            if (j<=i):
                b[i,j]=a[i,j]
            if (j>i):
                c[i,j]=a[i,j]
    res=(b,c)
    return res

def matr_m_matr(a,b,count):
    for i in range(0,count-1):
        for j in range(0,count-1):
            a[i,j]-=b[i,j]
    return a
    
def mpi(a,b,count,eps):
    result=method_yakobi(a,b,count-1)
    (alfa,beta)=result
    print('\nalfa=',alfa)
    print('\nbeta=',beta)
    if norma_matrix(alfa,count)<1:
        print('\nУсловие выполнено:')
        print('||alfa||=',norma_matrix(alfa,count),'<1\n')
    q=(norma_matrix(alfa,count)/(1-norma_matrix(alfa,count)))
    x=beta
    k=0
    while True:
      tmp=x
      x=matr_vect(alfa,tmp)
      x=vect_p_vect(beta,x)
      epsk=vect_m_vect(x,tmp)
      epsk=q*norma_vector(epsk)
      k+=1
      if epsk<eps:
        break
    print(k,' итераций')
    print(x)        
    return

def zeidel(a,b,count,eps):
    result=method_yakobi(a,b,count-1)
    (alfa,beta)=result
    print('\nalfa=',alfa)
    print('\nbeta=',beta)
    if norma_matrix(alfa,count)<1:
        print('\nУсловие выполнено:')
        print('||alfa||=',norma_matrix(alfa,count),'<1\n')
    q=(norma_matrix(alfa,count)/(1-norma_matrix(alfa,count)))
    x=beta
    k=0
    while True:
        tmp=x
        res=sum_razl(alfa,count-1)
        (bb,cc)=res
        ee=np.eye(count-1)
        dd=matr_m_matr(ee,bb,count)
        dd=linalg.inv(dd)
        x=vect_p_vect(matr_vect(dd,matr_vect(cc,x)),matr_vect(dd,beta))
        epsk=vect_m_vect(x,tmp)
        epsk=norma_vector(epsk)
        epsk=(norma_matrix(cc,count)/(1-norma_matrix(alfa,count)))*epsk
        k+=1
        if epsk<eps:
            break
    print(k,' итераций')
    print(x)       
    return

In [5]:
def maxel(a,count):
    maxelem=math.fabs(a[0,1])
    l=0
    m=1
    for i in range(0,count):
        for j in range(i+1,count):
            if (math.fabs(a[i,j])>maxelem) and (i<j):
                maxelem=math.fabs(a[i,j])
                l=i
                m=j
    return maxelem,l,m

def multi_2matr(a,b,count):
    c=np.zeros((count,count))
    for i in range(0,count):
        for j in range(0,count):
            for k in range(0,count):
                c[i,j]+=a[i,k]*b[k,j]
    return c

def off(a,l,m,count):
    summ=0
    for l in range(0,count):
        for m in range(l+1,count):
            summ=summ+(a[l,m]*a[l,m])
    return math.sqrt(summ)

def trans(a,count):
    result=maxel(a,count)
    (maxelem,l,m)=result
    if a[l,l]==a[m,m]:
        phi=math.pi/4
    else:
        phi=0.5*(math.atan(2*a[l,m]/(a[l,l]-a[m,m])))
    c=math.cos(phi)
    s=math.sin(phi)
    u=np.eye(count)
    u[l,l]=c
    u[l,m]=-s
    u[m,m]=c
    u[m,l]=s
    ut=np.eye(count)
    ut[l,l]=c
    ut[m,m]=c
    ut[l,m]=s
    ut[m,l]=-s
    res=(u,ut,l,m)
    return res
    
def rotation(a,count,eps):
    k=0
    v=np.eye(count)
    while True:
        result=trans(a,count)
        (u,ut,l,m)=result
        a=multi_2matr(multi_2matr(ut,a,count),u,count)
        v=multi_2matr(v,u,count)
        k+=1
        res=(a,v)
        if off(a,l,m,count)<eps:
            break
    print(k, 'итераций')       
    return res

In [6]:
def qr_dec(A,count):
    m, n = A.shape
    Q = np.eye(m)
    for i in range(n - (m == n)):
        H = np.eye(m)
        H[i:, i:] = make_householder(A[i:, i])
        Q = np.dot(Q, H)
        A = np.dot(H, A)
    return Q, A
 
def make_householder(a):
    v = a / (a[0] + np.copysign(np.linalg.norm(a), a[0]))
    v[0] = 1
    H = np.eye(a.shape[0])
    H -= (2 / np.dot(v, v)) * np.dot(v[:, None], v[None, :])
    return H

def qr_alg(A,n,eps):
    k = 0
    R = np.array(A)
    it = 0   
    while True:
        it += 1
        e_k = 0
        Q, A = qr_dec(A,n)
        A = np.dot(A, Q)
        print('\n',it,'итерация\nA = ',A)
        e_k = math.fabs(A[1,0])
        e_k_k = math.fabs(A[2,0])
        e_k_k_k = math.fabs(A[2,1])
        print('epsk = ',e_k_k)
        if e_k < eps or e_k_k < eps or e_k_k_k < eps:
           break           
    D = ((A[1][1] + A[2][2]) ** 2 - 4 * (A[1][1] * A[2][2] - A[1][2] * A[2][1]))
    l = []
    l.append("\nx1 = {}".format(A[0][0]))
    l.append("\nx2 = {}".format(((A[1][1] + A[2][2]) + D**0.5)/2))   
    l.append("\nx3 = {}".format(((A[1][1] + A[2][2]) - D**0.5)/2)) 
    print('\nСобмтвенные значения:')
    for i in range(len(l)):
            print(l[i] )
    return

def norm(vec, i, n):
    res = 0
    for j in range(i, n):
        res += vec[j] ** 2
    return sqrt(res)

def mult(a,n):
    first = np.zeros((n, n))
    first[0] = a
    return np.dot(np.transpose(first), first)

In [7]:
def main():
    print('Лабораторная работа 1\nВариант 2')
    
    while True:
        print('\nВыберите метод:\n1 - Алгоритм LU-разложнния матрицы\n2 - Метод прогонки\n3 - Метод простых итераций и метод Зейделя\n4 - Метод вращений\n5 - QR-алгоритм\n0 - выход')
        k=int(input())
        if k==0:
            break
        if k==1:
            f = open('input_lab1_1.txt')
            line = f.readlines()
            count=len(line)
            a=line[0].split()
            a=list(map(float,a))
            for i in range(1,count-1):
                tmp=line[i].split()
                tmp=list(map(float,tmp))
                a=np.row_stack((a,tmp))
            b=line[count-1].split()
            b=list(map(float,b))
            f.close()
            
            print('\nАлгоритм LU-разложения матрицы')    
            print('\nА= ',a)
            print('b= ',b)
            print('\nРешение СЛАУ:')
            print('x= ',linalg.solve(a,b))
            print('\nОпределитель матрицы:')
            print('det A= ',linalg.det(a))
            print('\nОбратная матрица:')
            print('A*= ',linalg.inv(a))
    
            res=LU(a,b)
            (u,l)=res
            print('\nU= ',u)
            print('\nL= ',l)   
            print('\nПроверка:')
            print('L*U= ',l.dot(u))        
            z=np.zeros(len(b))
            for i in range(0,len(b)):
                sum=0
                for j in range(0,i):
                    sum+=l[i,j]*z[j]
                z[i]=b[i]-sum
                
        if k==2:
            f = open('input_lab1_2.txt')
            line = f.readlines()
            a=line[0].split()
            a=list(map(int, a))
            b=line[1].split()
            b=list(map(int, b))
            c=line[2].split()
            c=list(map(int, c))
            d=line[3].split()
            d=list(map(int, d))
            f.close()
            
            print('\nМетод прогонки')
            print('\nРешение СЛАУ: x=',prog(a,b,c,d))
        
        if k==3:
            f = open('input_lab1_3.txt')
            line = f.readlines()
            count=len(line)
            a=line[0].split()
            a=list(map(float,a))
            for i in range(1,count-1):
                tmp=line[i].split()
                tmp=list(map(float,tmp))
                a=np.row_stack((a,tmp))
            b=line[count-1].split()
            b=list(map(float,b))
            f.close()
            
            print('\nА=',a)
            print('\nb=',b)
            print('\nВведите точность вычислений:')
            eps=float(input())
            print('eps=',eps)
            print('\nМетод простых итераций')
            mpi(a,b,count,eps)
            print('\nМетод Зейделя')
            zeidel(a,b,count,eps)
            
        if k==4:
            f = open('input_lab1_4.txt')
            line = f.readlines()
            count=len(line)
            a=line[0].split()
            a=list(map(float,a))
            for i in range(1,count):
                tmp=line[i].split()
                tmp=list(map(float,tmp))
                a=np.row_stack((a,tmp))
            f.close()
            
            print('\nМетод вращений')
            print('\nА=',a)
            print('\nВведите точность вычислений:')
            eps=float(input())
            print('eps=',eps)
            result=rotation(a,count,eps)
            (a,v)=result
            print('\nСобственные значения:')
            for i in range(0,count):
                print(a[i,i])
            print('\nСобственные векторы:')
            for i in range(0,count):
                sz=np.zeros(count)
                for j in range(0,count):
                    sz[j]=v[j,i]
                print(sz)
                
        if k==5:
            f = open('input_lab1_5.txt')
            line = f.readlines()
            n=len(line)
            a=line[0].split()
            a=list(map(float,a))
            for i in range(1,n):
                tmp=line[i].split()
                tmp=list(map(float,tmp))
                a=np.row_stack((a,tmp))
            f.close()
            
            print('\nQR-алгоритм')
            print('\nА=',a)
            print('\nВведите точность вычислений:')
            eps=float(input())
            (q1,r1) = linalg.qr(a)
            print('\nQR-разложение функцией из numpy: \n\nQnp = ',q1,'\n\nRnp = ',r1)
            (q,r) = qr_dec(a,n)
            print('\nQR-разложение: \n\nQ = ',q,'\n\nR = ',r)
            qr_alg(a,n,eps)
            
    return
main()

Лабораторная работа 1
Вариант 2

Выберите метод:
1 - Алгоритм LU-разложнния матрицы
2 - Метод прогонки
3 - Метод простых итераций и метод Зейделя
4 - Метод вращений
5 - QR-алгоритм
0 - выход
5

QR-алгоритм

А= [[-6. -4.  0.]
 [-7.  6. -7.]
 [-2. -6. -7.]]

Введите точность вычислений:
0.00001

QR-разложение функцией из numpy: 

Qnp =  [[-0.63599873  0.4706035  -0.61158644]
 [-0.74199852 -0.59065541  0.31711889]
 [-0.21199958  0.65548344  0.72484318]] 

Rnp =  [[ 9.43398113 -0.63599873  6.67798664]
 [ 0.         -9.35924706 -0.45379623]
 [ 0.          0.         -7.29373453]]

QR-разложение: 

Q =  [[-0.63599873  0.4706035  -0.61158644]
 [-0.74199852 -0.59065541  0.31711889]
 [-0.21199958  0.65548344  0.72484318]] 

R =  [[ 9.43398113e+00 -6.35998728e-01  6.67798664e+00]
 [ 8.48088668e-16 -9.35924706e+00 -4.53796228e-01]
 [-6.27026911e-16 -7.11803746e-16 -7.29373453e+00]]

 1 итерация
A =  [[-6.94382022  9.19263024 -1.13088901]
 [ 7.04075204  5.23063398 -3.29692517]
 [ 1.54626863 -4.780